In [70]:
def mysqlInsert(mapperInsertSet, feedcontrolInsetSet, filemetadataInsertSet):
    import mysql.connector
    from mysql.connector import Error
    from mysql.connector import errorcode
    import pandas as pd

    try:
        connection = mysql.connector.connect(host='10.3.2.13',
                                 database='etl',
                                 user='etlmysql',
                                 password='tata@123')
        
        mycursor = connection.cursor()
        retVal=''
        
        #Mapper table inserts
        if len(mapperInsertSet) > 0 :
            sql_insert_query = """ INSERT INTO etl.mapper (feedname, key_col, value_col) 
                               VALUES (%s,%s,%s) """        
            result  = mycursor.executemany(sql_insert_query, mapperInsertSet)
            connection.commit()
            retVal=str(mycursor.rowcount) + " Records inserted successfully into etl.mapper table"
        
        #Mapper table inserts
        if len(feedcontrolInsetSet) > 0 :
            sql_insert_query = """ INSERT INTO etl.feedcontrol (feedname,filename,dataformat,landingpath,rejectthreshold,processtype,headertrailerflag,rawzonepath,sourcetablename) 
                               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s) """        
            result  = mycursor.executemany(sql_insert_query, feedcontrolInsetSet)
            connection.commit()
            retVal=retVal + '\n' + str(mycursor.rowcount) + " Records inserted successfully into etl.feedcontrol table"

        #Mapper table inserts
        if len(filemetadataInsertSet) > 0 :
            sql_insert_query = """ INSERT INTO etl.filemetadata (feedname,filename,filetype,filedelimiter) 
                               VALUES (%s,%s,%s,%s) """        
            result  = mycursor.executemany(sql_insert_query, feedcontrolInsetSet)
            connection.commit()
            retVal=retVal + '\n' + str(mycursor.rowcount) + " Records inserted successfully into etl.filemetadata table"
            
        return retVal
    except mysql.connector.Error as error :
        retVal ="Failed inserting records into metatdata tables" + error
        return retVal
    finally:
        #closing database connection.
        if(connection.is_connected()):
            mycursor.close()
            connection.close()
            #print("connection is closed")

In [74]:
dbname='abc'
user='abc'
feedname='TEST'
mapperSet = [ (feedname,'USER',user) , (feedname,'DBNAME',dbname) ]
#feedcontrolSet = [(feedname, '', 'MYSQL', '', 0, '', 'N', '', 'abc' )]
feedcontrolSet=[]
filemetadataInsertSet = [('TEst3', 'abc.dat', 'DELIMITED', ',')]
ret=mysqlInsert(mapperSet, feedcontrolSet, filemetadataInsertSet)
print(ret)

2 Records inserted successfully into etl.mapper table
2 Records inserted successfully into etl.filemetadata table


In [96]:
import json
import pandas as pd
connection = mysql.connector.connect(host='10.3.2.13',
                         database='etl',
                         user='etlmysql',
                         password='tata@123')

#mycursor = connection.cursor()
#mycursor.execute("SELECT * FROM etl.feedcontrol where feedname = 'TEst3'  ")
#row_headers=[x[0] for x in mycursor.description] #this will extract row headers
#rv = mycursor.fetchall()
#json_data=[]
#for result in rv:
#    json_data.append(dict(zip(row_headers,result)))
#return json.dumps(json_data)
#myresult = mycursor.fetchall()
#print(json.dumps(json_data))

df = pd.read_sql("SELECT * FROM etl.feedcontrol where feedname = 'TEst3'  ", connection)
df.head()

,feedname,filename,dataformat,landingpath,rejectthreshold,processtype,headertrailerflag,sourcetablename
0,TEst3,abc.dat,DELIMITED,/home/landing,0,,N,abc
1,TEst3,abc.dat,DELIMITED,/home/landing,0,,N,abc
2,TEst3,abc.dat,DELIMITED,/home/landing,0,,N,abc
3,TEst3,abc.dat,DELIMITED,/home/landing,0,,N,abc
4,TEst3,abc.dat,DELIMITED,/home/landing,0,,N,abc


In [1]:
import sys
import logging
import json
import os, time  #import statements
from datetime import datetime
import mysql.connector
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from stat import *
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import StringType
import pyspark.sql.functions as f
SparkContext.setSystemProperty("hive.metastore.uris", "thrift://10.3.2.20:9083") # comment
sparkSession = (SparkSession.builder.appName('pyspark-to-load-tables-hive').enableHiveSupport().getOrCreate())
spark = SparkSession.builder.appName('changeColNames').getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/jars/hadoop-aws-2.7.3.jar /opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/jars/aws-java-sdk-1.7.4.jar pyspark-shell'

#SparkContext.hadoopConfiguration.set("fs.s3n.awsAccessKeyId", '2YQT684ZYDIDXC6K8NHP')
#SparkContext.hadoopConfiguration.set("fs.s3n.awsSecretAccessKey", '6n0BcktZJB7uVLUM9BioDsFgzD3B+K+ctwxOZije')
#SparkContext.hadoopConfiguration.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")


In [6]:
#lines = sc.textFile("file:///home/etl/praveen/commands.txt")
#lines.take(1)
#df.write.format('csv').save(rawzonepath, mode='append')
filename="file:///home/etl/test.dat"
colsep='|'
hdfsFile='hdfs://10.3.2.13:8020/user/hadoop/rawzone/test.dat'
df=spark.read.load(filename, format='csv', sep=colsep)
df.show()
df.write.format('csv').save(hdfsFile, mode='append')

+----+-----+----+------+----+----+
| _c0|  _c1| _c2|   _c3| _c4| _c5|
+----+-----+----+------+----+----+
|   1|   KA|  IN|     4|   5|null|
|asds|sdfdf|sddd|dsfdsf|dffd|null|
|   3|    5|   6|  fgfg|  44|null|
+----+-----+----+------+----+----+

